In [1]:
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [2]:
import gc
gc.collect()

42

In [3]:
# torch.cuda.empty_cache()

In [4]:
class Opt:
    def __init__(self):
        self.n_epochs = 200
        self.batch_size = 16384 # 128*128 img_size*img_size !!!!!!!!!!!!!!!!!debugger wayne931121: this line don't use 1, or batch_size/(batch_size-1) will inf
        self.lr = 0.0002
        self.b1 = 0.5
        self.b2 = 0.999
        self.n_cpu = 8
        self.latent_dim  = 62
    #    self.n_classes = 10
        self.img_size = 128
        self.channels = 3
        self.sample_interval = 400

In [5]:
opt = Opt()

In [6]:
img_shape = (opt.channels, opt.img_size, opt.img_size)

cuda = True if torch.cuda.is_available() else False


def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = opt.img_size // 4
        self.l1 = nn.Sequential(nn.Linear(opt.latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, opt.channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise):
        out = self.l1(noise)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        # Upsampling
        self.down = nn.Sequential(nn.Conv2d(opt.channels, 64, 3, 2, 1), nn.ReLU())
        # Fully-connected layers
        self.down_size = opt.img_size // 2
        down_dim = 64 * (opt.img_size // 2) ** 2

        self.embedding = nn.Linear(down_dim, 32)

        self.fc = nn.Sequential(
            nn.BatchNorm1d(32, 0.8),
            nn.ReLU(inplace=True),
            nn.Linear(32, down_dim),
            nn.BatchNorm1d(down_dim),
            nn.ReLU(inplace=True),
        )
        # Upsampling
        self.up = nn.Sequential(nn.Upsample(scale_factor=2), nn.Conv2d(64, opt.channels, 3, 1, 1))

    def forward(self, img):
        out = self.down(img)
        embedding = self.embedding(out.view(out.size(0), -1))
        out = self.fc(embedding)
        out = self.up(out.view(out.size(0), 64, self.down_size, self.down_size))
        return out, embedding


# Reconstruction loss of AE
pixelwise_loss = nn.MSELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    pixelwise_loss.cuda()

# Initialize weights
test = generator.apply(weights_init_normal)
test = discriminator.apply(weights_init_normal)

In [7]:
# https://discuss.pytorch.org/t/error-expected-more-than-1-value-per-channel-when-training/26274
test = generator.eval()
test = discriminator.eval()

In [8]:
p = r"C:\Users\原神\Downloads\cats"+"\\"
name = "cats"

In [9]:
# import os, sys
# from PIL import Image

# size = (128,128)

# for infile in os.listdir(p+"val\\"+name):
#         im = Image.open(p+"val\\"+name+"\\"+infile)
#         im = im.resize(size)
#         im = im.convert("RGB")
#         os.remove(p+"val\\"+name+"\\"+infile)
#         im.save(p+"val\\"+name+"\\"+infile.replace("png","jpg"), "JPEG")

# for infile in os.listdir(p+"train\\"+name):
#         im = Image.open(p+"train\\"+name+"\\"+infile)
#         im = im.resize(size)
#         im = im.convert("RGB")
#         os.remove(p+"train\\"+name+"\\"+infile)
#         im.save(p+"train\\"+name+"\\"+infile.replace("png","jpg"), "JPEG")

In [10]:
dataset = datasets.ImageFolder(root=p,
                               transform = transforms.Compose(
                                   [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]),
                              )

In [11]:
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=opt.batch_size,
    shuffle=True,
)

In [12]:
generator = Generator()
if cuda:
    generator.cuda()
generator.apply(weights_init_normal)
test = generator.eval()

In [13]:
#opt.lr = 0.0002

optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

#debugger wayne931121
def pullaway_loss(embeddings):
    norm = torch.sqrt(torch.sum(embeddings ** 2, -1, keepdim=True))
    normalized_emb = embeddings / norm
    similarity = torch.matmul(normalized_emb, normalized_emb.transpose(1, 0))
    batch_size = embeddings.size(0)
    if (batch_size * (batch_size - 1)):
        loss_pt = (torch.sum(similarity) - batch_size) / (batch_size * (batch_size - 1))
    else:
        loss_pt = (torch.sum(similarity) - batch_size)  / 64*63
    return loss_pt

In [14]:
opt.n_epochs = 5000

In [15]:
# optimizer_G.param_groups[0]['lr'] = 1e-4
# optimizer_D.param_groups[0]['lr'] = 1e-4

In [16]:
opt.sample_interval = 100

In [17]:
batches_done = 0

In [19]:
lambda_pt = 0.1
margin = max(1, opt.batch_size / 64.0)

for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)
        recon_imgs, img_embeddings = discriminator(gen_imgs)

        # Loss measures generator's ability to fool the discriminator
        g_loss = pixelwise_loss(recon_imgs, gen_imgs.detach()) + lambda_pt * pullaway_loss(img_embeddings)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_recon, _ = discriminator(real_imgs)
        fake_recon, _ = discriminator(gen_imgs.detach())

        d_loss_real = pixelwise_loss(real_recon, real_imgs)
        d_loss_fake = pixelwise_loss(fake_recon, gen_imgs.detach())

        d_loss = d_loss_real
        if (margin - d_loss_fake.data).item() > 0:
            d_loss += margin - d_loss_fake

        d_loss.backward()
        optimizer_D.step()

        # --------------
        # Log Progress
        # --------------

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done += 1
        if batches_done % opt.sample_interval == 0:
            save_image(gen_imgs.data[:25], r"C:\Users\原神\Downloads\result1"+"\\%d.png" % batches_done, nrow=5, normalize=True)


[Epoch 0/5000] [Batch 0/1] [D loss: 119.346649] [G loss: 306.440948]
[Epoch 1/5000] [Batch 0/1] [D loss: 0.345574] [G loss: 653.231079]
[Epoch 2/5000] [Batch 0/1] [D loss: 0.344775] [G loss: 263.407562]
[Epoch 3/5000] [Batch 0/1] [D loss: 68.100685] [G loss: 188.342545]
[Epoch 4/5000] [Batch 0/1] [D loss: 0.345550] [G loss: 511.495880]
[Epoch 5/5000] [Batch 0/1] [D loss: 1.965192] [G loss: 769.536865]
[Epoch 6/5000] [Batch 0/1] [D loss: 2.639212] [G loss: 674.649414]
[Epoch 7/5000] [Batch 0/1] [D loss: 0.624839] [G loss: 577.474121]
[Epoch 8/5000] [Batch 0/1] [D loss: 0.345925] [G loss: 1251.442261]
[Epoch 9/5000] [Batch 0/1] [D loss: 0.345762] [G loss: 528.769348]
[Epoch 10/5000] [Batch 0/1] [D loss: 188.629684] [G loss: 67.814919]
[Epoch 11/5000] [Batch 0/1] [D loss: 59.742126] [G loss: 537.526306]
[Epoch 12/5000] [Batch 0/1] [D loss: 0.595457] [G loss: 370.031708]
[Epoch 13/5000] [Batch 0/1] [D loss: 0.345127] [G loss: 1118.203491]
[Epoch 14/5000] [Batch 0/1] [D loss: 0.344782] [G l

In [18]:
# img_embeddings

In [19]:
# gen_imgs

In [20]:
# generator.l1(z)

In [21]:
# generator.l1(z)

In [22]:
        # gen_imgs = generator(z)
        # recon_imgs, img_embeddings = discriminator(gen_imgs)

In [23]:
# z

In [24]:
# gen_imgs

In [21]:
img_embeddings.size()

torch.Size([19, 32])

In [26]:
# pullaway_loss(img_embeddings)

In [27]:
# pixelwise_loss(recon_imgs, gen_imgs.detach())

In [28]:
# embeddings = img_embeddings
# norm = torch.sqrt(torch.sum(embeddings ** 2, -1, keepdim=True))
# norm

In [29]:
# normalized_emb = embeddings / norm
# normalized_emb

In [30]:
# similarity = torch.matmul(normalized_emb, normalized_emb.transpose(1, 0))
# similarity

In [31]:
# batch_size = embeddings.size(0)
# batch_size

In [32]:
# (torch.sum(similarity) - batch_size)

In [33]:
# (batch_size * (batch_size - 1))

In [34]:
# batch_size

In [37]:
# generator.init_size

In [36]:
# opt.img_size // 4

In [37]:
# z

In [38]:
# h = nn.Sequential(nn.Linear(opt.latent_dim, 128 * 32 ** 2))
# h.cuda()
# h(z)

In [39]:
# generator.l1 = nn.Sequential(nn.Linear(opt.latent_dim, 128 * generator.init_size ** 2))
# generator.l1.cuda()

In [40]:
# g = Generator()
# g.cuda()

In [42]:
# g.l1(z)

In [43]:
# g.eval()

In [44]:
# g.apply(weights_init_normal)

In [45]:
# generator.l1.cuda()

In [46]:
# generator.l1(z)

In [47]:
# generator.l1(z)

In [48]:
# generator.__call__

In [49]:
# generator(z)

In [50]:
# opt.latent_dim

In [51]:
# imgs.shape

In [52]:
# np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))

In [53]:
# -6.47948121e-01

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# sample_image(n_row=opt.n_classes, batches_done=batches_done)

In [ ]:
gp = r"C:\Users\TEST\generator_001.pth"
dp = r"C:\Users\TEST\discriminator_001.pth"

In [ ]:
torch.save(generator.state_dict(), gp)
torch.save(discriminator.state_dict(), dp)

In [ ]:
#generate a random image
sample_image(n_row=1, batches_done=-1)

In [ ]:
discriminator.load_state_dict(torch.load(dp, weights_only=True))
discriminator.eval()

In [ ]:
generator.load_state_dict(torch.load(gp, weights_only=True))
generator.eval()